In [1]:
import os
import cv2

# Define the dataset path
dataset_path = r"C:\Users\Lenovo\OneDrive - Alexandria University\Desktop\Projects\TheftClassification\Shop DataSet"
non_lifters_path = os.path.join(dataset_path, 'non shop lifters')
lifters_path = os.path.join(dataset_path, 'shop lifters')

# Function to get video info (number of frames, dimensions)
def get_video_info(video_path):
    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        return None
    frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    cap.release()
    return frame_count, frame_width, frame_height

# Explore videos in both classes
def explore_videos(directory):
    video_files = os.listdir(directory)
    video_info = []
    for video in video_files:
        video_path = os.path.join(directory, video)
        info = get_video_info(video_path)
        if info:
            frame_count, frame_width, frame_height = info
            video_info.append({
                'file': video,
                'frames': frame_count,
                'width': frame_width,
                'height': frame_height
            })
    return video_info

# Explore the non-lifters videos
print("Non-shoplifters:")
non_lifters_info = explore_videos(non_lifters_path)
for video in non_lifters_info:
    print(video)

# Explore the shoplifters videos
print("\nShoplifters:")
lifters_info = explore_videos(lifters_path)
for video in lifters_info:
    print(video)

# Print out some basic statistics
print(f"\nTotal non-shoplifters videos: {len(non_lifters_info)}")
print(f"Total shoplifters videos: {len(lifters_info)}")

Non-shoplifters:
{'file': 'shop_lifter_n_0.mp4', 'frames': 450, 'width': 704, 'height': 576}
{'file': 'shop_lifter_n_0_1.mp4', 'frames': 450, 'width': 704, 'height': 576}
{'file': 'shop_lifter_n_1.mp4', 'frames': 450, 'width': 704, 'height': 576}
{'file': 'shop_lifter_n_10.mp4', 'frames': 475, 'width': 704, 'height': 576}
{'file': 'shop_lifter_n_100.mp4', 'frames': 400, 'width': 704, 'height': 576}
{'file': 'shop_lifter_n_100_1.mp4', 'frames': 400, 'width': 704, 'height': 576}
{'file': 'shop_lifter_n_101.mp4', 'frames': 300, 'width': 704, 'height': 576}
{'file': 'shop_lifter_n_101_1.mp4', 'frames': 300, 'width': 704, 'height': 576}
{'file': 'shop_lifter_n_102.mp4', 'frames': 275, 'width': 704, 'height': 576}
{'file': 'shop_lifter_n_102_1.mp4', 'frames': 275, 'width': 704, 'height': 576}
{'file': 'shop_lifter_n_103.mp4', 'frames': 275, 'width': 704, 'height': 576}
{'file': 'shop_lifter_n_103_1.mp4', 'frames': 275, 'width': 704, 'height': 576}
{'file': 'shop_lifter_n_104.mp4', 'frames': 

In [2]:
import os
import shutil
from sklearn.model_selection import train_test_split

# Directories with your videos
shop_lifters_dir = r"C:\Users\Lenovo\OneDrive - Alexandria University\Desktop\Projects\TheftClassification\Shop DataSet\shop lifters"
non_shop_lifters_dir = r"C:\Users\Lenovo\OneDrive - Alexandria University\Desktop\Projects\TheftClassification\Shop DataSet\non shop lifters"

# Directories for split data
preprocessed_train_dir = r"C:\Users\Lenovo\OneDrive - Alexandria University\Desktop\Projects\TheftClassification\Preprocessed2\train"
preprocessed_valid_dir = r"C:\Users\Lenovo\OneDrive - Alexandria University\Desktop\Projects\TheftClassification\Preprocessed2\valid"
preprocessed_test_dir = r"C:\Users\Lenovo\OneDrive - Alexandria University\Desktop\Projects\TheftClassification\Preprocessed2\test"

# Create directories if not already present
for folder in [preprocessed_train_dir, preprocessed_valid_dir, preprocessed_test_dir]:
    os.makedirs(folder + '/shop_lifters', exist_ok=True)
    os.makedirs(folder + '/non_shop_lifters', exist_ok=True)

# Get video file paths and labels
shoplifters_video_paths = [os.path.join(shop_lifters_dir, f) for f in os.listdir(shop_lifters_dir) if f.endswith('.mp4')]
non_shoplifters_video_paths = [os.path.join(non_shop_lifters_dir, f) for f in os.listdir(non_shop_lifters_dir) if f.endswith('.mp4')]

# Labels (1 for shoplifters, 0 for non-shoplifters)
shoplifters_labels = [1] * len(shoplifters_video_paths)
non_shoplifters_labels = [0] * len(non_shoplifters_video_paths)

# Combine video paths and labels
video_paths = shoplifters_video_paths + non_shoplifters_video_paths
labels = shoplifters_labels + non_shoplifters_labels

# Split into train + temp (for valid/test), stratify by labels
X_train, X_temp, y_train, y_temp = train_test_split(video_paths, labels, test_size=0.3, stratify=labels, random_state=42)

# Split temp into validation and test, stratify by labels
X_valid, X_test, y_valid, y_test = train_test_split(X_temp, y_temp, test_size=0.5, stratify=y_temp, random_state=42)

# Function to move files to the respective directories
def move_files(file_paths, labels, target_dir):
    for file_path, label in zip(file_paths, labels):
        if label == 1:  # Shoplifters
            target_folder = os.path.join(target_dir, 'shop_lifters')
        else:  # Non-shoplifters
            target_folder = os.path.join(target_dir, 'non_shop_lifters')
        shutil.copy(file_path, target_folder)  # Copy video to target folder

# Move videos to train/validation/test directories
move_files(X_train, y_train, preprocessed_train_dir)
move_files(X_valid, y_valid, preprocessed_valid_dir)
move_files(X_test, y_test, preprocessed_test_dir)

print("Train, validation, and test split completed.")


Train, validation, and test split completed.


In [3]:
import os
import cv2
import numpy as np
import random
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import Sequence
import tensorflow as tf

def adjust_frame_count(frames, target_frame_count=100):
    frame_count = len(frames)
    
    if frame_count == target_frame_count:
        return frames
    elif frame_count > target_frame_count:
        # Use uniform sampling to preserve overall structure
        indices = np.linspace(0, frame_count - 1, target_frame_count, dtype=int)
        return [frames[i] for i in indices]
    else:
        # Interpolate missing frames
        repeats = np.ceil(target_frame_count / frame_count).astype(int)
        indices = np.arange(frame_count).repeat(repeats)[:target_frame_count]
        return [frames[i] for i in indices]

In [4]:
# Function to apply consistent augmentation for the same video
def augment_frame(frame, augmentation_params):
    if augmentation_params.get("flip", False):
        frame = cv2.flip(frame, 1)  # Horizontal flip

    angle = augmentation_params.get("rotation", 0)
    if angle != 0:
        h, w = frame.shape[:2]
        M = cv2.getRotationMatrix2D((w // 2, h // 2), angle, 1)
        frame = cv2.warpAffine(frame, M, (w, h))

    brightness = augmentation_params.get("brightness", 0)
    contrast = augmentation_params.get("contrast", 1.0)
    frame = cv2.convertScaleAbs(frame, alpha=contrast, beta=brightness)

    return frame

# Generate augmentation parameters for each video
def generate_augmentation_params():
    return {
        "flip": random.random() > 0.5,
        "rotation": random.uniform(-7, 7),
        "brightness": random.randint(-20, 20),
        "contrast": random.uniform(0.85, 1.15),
    }

In [5]:
# Preprocess video with optional augmentation for training data
def preprocess_video(video_path, desired_size=(128, 128), target_frames=100, augment=False):
    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        print(f"Failed to open video: {video_path}")
        return None

    if augment:
        augmentation_params = generate_augmentation_params()
    else:
        augmentation_params = None

    frames = []
    while True:
        ret, frame = cap.read()
        if not ret:
            break
        resized_frame = cv2.resize(frame, desired_size)
        if augment:
            resized_frame = augment_frame(resized_frame, augmentation_params)
        frames.append(resized_frame)

    cap.release()
    frames = adjust_frame_count(frames, target_frames)
    frames = np.array(frames) / 255.0  # Normalize to [0, 1]

    return frames

In [6]:
class VideoDataGenerator(Sequence):
    def __init__(self, video_paths, labels, batch_size, target_frames=100, desired_size=(128, 128), augment=False, shuffle=True):
        self.video_paths = video_paths
        self.labels = labels
        self.batch_size = batch_size
        self.target_frames = target_frames
        self.desired_size = desired_size
        self.augment = augment
        self.shuffle = shuffle
        self.on_epoch_end()
    
    def __len__(self):
        return int(np.floor(len(self.video_paths) / self.batch_size))
    
    def __getitem__(self, index):
        batch_indexes = self.indexes[index * self.batch_size:(index + 1) * self.batch_size]
        batch_video_paths = [self.video_paths[i] for i in batch_indexes]
        batch_labels = [self.labels[i] for i in batch_indexes]
        X, y = self.__data_generation(batch_video_paths, batch_labels)
        return X, y
    
    def on_epoch_end(self):
        self.indexes = np.arange(len(self.video_paths))
        if self.shuffle:
            np.random.shuffle(self.indexes)
    
    def __data_generation(self, batch_video_paths, batch_labels):
        X = np.empty((self.batch_size, self.target_frames, *self.desired_size, 3))
        y = np.empty((self.batch_size), dtype=int)
        for i, (video_path, label) in enumerate(zip(batch_video_paths, batch_labels)):
            X[i,] = preprocess_video(video_path, desired_size=self.desired_size, target_frames=self.target_frames, augment=self.augment)
            y[i] = label
        return X, y

In [7]:
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from sklearn.utils.class_weight import compute_class_weight

# Parameters for the generator
batch_size = 4
target_frames = 100
desired_size = (128, 128)

# Create data generators
train_generator = VideoDataGenerator(X_train, y_train, batch_size=batch_size, target_frames=target_frames, desired_size=desired_size, augment=True)  # Augmentation ON
valid_generator = VideoDataGenerator(X_valid, y_valid, batch_size=batch_size, target_frames=target_frames, desired_size=desired_size, augment=False)  # Augmentation OFF
test_generator = VideoDataGenerator(X_test, y_test, batch_size=batch_size, target_frames=target_frames, desired_size=desired_size, augment=False)  # Augmentation OFF

# Calculate class weights to handle class imbalance
class_weights = compute_class_weight(class_weight='balanced', classes=np.unique(y_train), y=y_train)
class_weight_dict = {0: class_weights[0], 1: class_weights[1]}


In [8]:
from sklearn.metrics import precision_score, recall_score
import numpy as np

In [9]:
import tensorflow as tf
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.layers import TimeDistributed, LSTM, Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint
from sklearn.metrics import precision_score, recall_score
import numpy as np

from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import TimeDistributed, LSTM, Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras import Input, Model

def create_video_classification_model(input_shape, num_classes=2):
    # Load pre-trained MobileNetV2 without top layers
    base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=input_shape[1:])
    
    # Freeze the base model layers to prevent fine-tuning
    base_model.trainable = False
    
    # TimeDistributed wrapper applies the MobileNet to each video frame
    inputs = Input(shape=input_shape)
    x = TimeDistributed(base_model)(inputs)
    x = TimeDistributed(GlobalAveragePooling2D())(x)  # Pooling across spatial dimensions for each frame
    
    # LSTM to capture temporal relationships between frames
    x = LSTM(128, return_sequences=True)(x)  # Reduce LSTM units to decrease parameters
    x = LSTM(64)(x)
    
    # Fully connected layer and classification output
    x = Dense(64, activation='relu')(x)
    x = Dropout(0.5)(x)
    outputs = Dense(num_classes, activation='softmax')(x)
    
    model = Model(inputs, outputs)
    return model


In [10]:
# Step 2: Create function to evaluate model performance
def evaluate_model(model, generator, dataset_name):
    """Function to evaluate model's accuracy, precision, and recall on a given dataset (train/val/test)."""
    y_true = []
    y_pred = []
    
    # Loop through the data generator and predict
    for X_batch, y_batch in generator:
        preds = model.predict(X_batch)
        y_true.extend(y_batch)
        y_pred.extend(np.argmax(preds, axis=1))
    
    y_true = np.array(y_true)
    y_pred = np.array(y_pred)
    
    accuracy = np.mean(y_true == y_pred)
    precision = precision_score(y_true, y_pred, average='binary')
    recall = recall_score(y_true, y_pred, average='binary')

    print(f"{dataset_name} Accuracy: {accuracy:.4f}, Precision: {precision:.4f}, Recall: {recall:.4f}")
    return accuracy, precision, recall

# Step 3: Define callbacks (EarlyStopping, ReduceLROnPlateau, and ModelCheckpoint)
early_stopping = EarlyStopping(
    monitor='val_loss', 
    patience=5, 
    restore_best_weights=True
)

reduce_lr_on_plateau = ReduceLROnPlateau(
    monitor='val_loss', 
    factor=0.2, 
    patience=3, 
    min_lr=1e-6
)


In [11]:
# ModelCheckpoint to save the model at the best validation accuracy
model_checkpoint = ModelCheckpoint(
    'best_mobile_lstm_model.keras',  # Filepath to save the best model
    monitor='val_accuracy', 
    save_best_only=True, 
    mode='max'
)

callbacks = [early_stopping, reduce_lr_on_plateau, model_checkpoint]

# Step 4: Define input shape and compile the model
input_shape = (100, 128, 128, 3)  # (frames, height, width, channels)
model = create_video_classification_model(input_shape)
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])


In [12]:
model.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer_1 (InputLayer)           │ (None, 100, 128, 128, 3)    │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ time_distributed (TimeDistributed)   │ (None, 100, 4, 4, 1280)     │       2,257,984 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ time_distributed_1 (TimeDistributed) │ (None, 100, 1280)           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm (LSTM)                          │ (None, 100, 128)            │         721,408 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_1 (LSTM)                        │ (None, 64)                  │          49,408 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 64)                  │           4,160 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 64)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 2)                   │             130 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 3,033,090 (11.57 MB)

 Trainable params: 775,106 (2.96 MB)

 Non-trainable params: 2,257,984 (8.61 MB)

In [13]:

# Step 5: Train the ResNet + LSTM model
print("Training ResNet + LSTM model...")
history1 = model.fit(
    train_generator,  # Assumes you've defined train_generator earlier
    validation_data=valid_generator,  # Assumes you've defined valid_generator earlier
    epochs=20,
    class_weight=class_weight_dict,  # Assumes you've calculated class weights earlier
    callbacks=callbacks  # Add callbacks here
)


Training ResNet + LSTM model...
Epoch 1/20


C:\ProgramData\anaconda3\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


149/149 ━━━━━━━━━━━━━━━━━━━━ 1664s 9s/step - accuracy: 0.4882 - loss: 0.7689 - val_accuracy: 0.3750 - val_loss: 0.7098 - learning_rate: 0.0010
Epoch 2/20
149/149 ━━━━━━━━━━━━━━━━━━━━ 1595s 11s/step - accuracy: 0.5298 - loss: 0.6937 - val_accuracy: 0.6250 - val_loss: 0.6351 - learning_rate: 0.0010
Epoch 3/20
149/149 ━━━━━━━━━━━━━━━━━━━━ 1582s 11s/step - accuracy: 0.5517 - loss: 0.6820 - val_accuracy: 0.8438 - val_loss: 0.3929 - learning_rate: 0.0010
Epoch 4/20
149/149 ━━━━━━━━━━━━━━━━━━━━ 1580s 11s/step - accuracy: 0.7137 - loss: 0.5863 - val_accuracy: 0.8281 - val_loss: 0.4227 - learning_rate: 0.0010
Epoch 5/20
149/149 ━━━━━━━━━━━━━━━━━━━━ 1676s 11s/step - accuracy: 0.7873 - loss: 0.5080 - val_accuracy: 0.8594 - val_loss: 0.3991 - learning_rate: 0.0010
Epoch 6/20
149/149 ━━━━━━━━━━━━━━━━━━━━ 1815s 12s/step - accuracy: 0.7825 - loss: 0.5155 - val_accuracy: 0.9141 - val_loss: 0.3310 - learning_rate: 0.0010
Epoch 7/20
149/149 ━━━━━━━━━━━━━━━━━━━━ 1839s 12s/step - accuracy: 0.8092 - loss: 

In [26]:
model.save("best_model_LSTM_MOBILE.keras")

In [30]:
# Evaluate the model on the test data
results = model.evaluate(test_generator)

# Assuming you compiled the model with 'loss' and 'accuracy' as metrics, results[0] is the loss and results[1] is accuracy.
print(f"Test Loss: {results[0]:.4f}")
print(f"Test Accuracy: {results[1] * 100:.2f}%")

32/32 ━━━━━━━━━━━━━━━━━━━━ 291s 9s/step - accuracy: 0.9540 - loss: 0.1449
Test Loss: 0.2216
Test Accuracy: 92.19%
